## User defined variables

In [10]:
## Paths for the the story data and FMRI data
grids_path = "../data/story_data/grids_huge.jbl"
trfiles_path = "../data/story_data/trfiles_huge.jbl"

## Paths for the topic encoder
model_dir = "../models"
embeddings_dir = "../embeddings"

## If there are any pre-trained models you'd like to use
model_path = None

# If embeddings have been pre-computed
embeddings_path = None

# Setups and Downloads

In [2]:
#### Dependencies ####
import numpy as np
import logging
import os
import sys
import time
import joblib
import matplotlib.pyplot as plt
import torch
import json
import cortex # This dependency is pycortex, which enables the plotting of flatmaps. It can be disabled.
from cvxopt import matrix, solvers # Only necessary for the stacked model.
from transformers import AutoTokenizer, AutoModelForCausalLM # Only necessary for feature extraction.
import subprocess
from tqdm.autonotebook import tqdm, trange

# Repository imports
from ridge_utils.ridge import bootstrap_ridge
import ridge_utils.npp
from ridge_utils.util import make_delayed
from ridge_utils.dsutils import make_word_ds
from ridge_utils.DataSequence import DataSequence
from ridge_utils.tokenization_helpers import generate_efficient_feat_dicts_opt
from ridge_utils.tokenization_helpers import convert_to_feature_mats_opt

# Topic model imports
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer # For generating embeddings
from sklearn.decomposition import PCA # To speed up the UMAP
from sklearn.feature_extraction.text import CountVectorizer 
from bertopic.vectorizers import ClassTfidfTransformer
try:
    import en_core_web_sm
except ModuleNotFoundError:
    !python -m spacy download en_core_web_sm
    import en_core_web_sm
from bertopic.representation import PartOfSpeech, KeyBERTInspired, MaximalMarginalRelevance

## Download cortex data
Here we automate the download of the brain models from open-neuro. It uses curl to download the files, and then sets the cortex path to be the correct location for us.

In [3]:
pycortex_download_script = "../ds003020-2.2.0.sh"
pycortex_dir = '../pycortex-db'

# Select which subjects to download (full list is ['UTS01', 'UTS02','UTS03','UTS04','UTS05','UTS06','UTS07','UTS08'] ) 
subjects = ['UTS01', 'UTS02','UTS03']

with open(pycortex_download_script, 'r') as f:
    pbar = tqdm(f)
    for line in pbar:
        if 'derivative/pycortex-db/UTS' in line:
            for subject in subjects:
                if subject in line:
                    # Construct the output command
                    output_command = line.replace(' derivative/pycortex-db/', ' ' + pycortex_dir + os.sep)
                    
                    # Extract the output file path from the curl command
                    # Assuming the output path is specified with -o option in the curl command
                    parts = output_command.split()
                    output_file_path = None
                    if '-o' in parts:
                        output_file_index = parts.index('-o') + 1
                        output_file_path = parts[output_file_index]
                    
                    # Check if the file exists
                    if output_file_path and not os.path.exists(output_file_path):
                        subprocess.run(output_command, shell=True)
                    else:
                        pbar.set_description(f"File {output_file_path} already exists. Skipping download.")

# This is your new filestore path
new_filestore_path = os.path.join(os.getcwd(), pycortex_dir)
cortex.options.config.set('basic', 'filestore', new_filestore_path)
# Set the new filestore path
cortex.db.filestore = cortex.options.config.get('basic', 'filestore')
cortex.db.reload_subjects()
cortex.db

0it [00:00, ?it/s]

Pycortex database
  Subjects:
   UTS01
   UTS02
   UTS03

## GPU Setup

Sets up the GPU if there is one there. Biggest benefit will be on CUDA systems, some benefits exist for MacOS 

In [4]:
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use MPS if available
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Fallback to CUDA or CPU

# The submodules of the topic model

## Purpose and Text Description
The topic model is being used to cluster small chunks of text into specific topics. The topic model we're using is BERTopic model. The overall flow of the model is as below:

1. [Embeddings](####Embeddings)
2. [Dimensionality Reduction](###Dimensionality-Reduction)
3. [Clustering](###Clustering)

Once the clusters are defined, we can then go further by:

4. [Vectorizer](###Vectorizer)
5. [Weighting](###Weighting)
6. [Topic Representation](###Topic-Representation)

If you can't be bothered and just want to see the code, it's here:[TLDR Setup](###Setup)

Parts 1-3 are required for first training the model. We can then fine tune and refine the clusters after the fact from 4-6.

### Clustering

#### Embeddings
`embedding_model`

In [11]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path = "all-MiniLM-L6-v2")

/Users/20759193/anaconda3/envs/fmri/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


KeyboardInterrupt: 

We start by converting our documents to numerical representations. Although there are many methods for doing so the default in BERTopic is sentence-transformers. These models are often optimized for semantic similarity which helps tremendously in our clustering task. Moreover, they are great for creating either document- or sentence-embeddings.
In BERTopic, you can choose any sentence-transformers model but two models are set as defaults:

```"all-MiniLM-L6-v2"```

```"paraphrase-multilingual-MiniLM-L12-v2"```

The first is an English language model trained specifically for semantic similarity tasks which works quite well for most use cases. The second model is very similar to the first with one major difference being that the multilingual models work for 50+ languages. This model is quite a bit larger than the first and is only selected if you select any language other than English.

New embedding models are released frequently and their performance keeps getting better. To keep track of the best embedding models out there, you can visit the [MTEB leaderboard](https://huggingface.co/spaces/mteb/leaderboard). It is an excellent place for selecting the embedding that works best for you. For example, if you want the best of the best, then the top 5 models might the place to look.

The basic premise is that we feed in a text string (of unspecified length), and the string is then passed to a sentence transformer. Sentence transformers are models that encode sentences into fixed-length numerical representations, also known as sentence embeddings. These embeddings capture the semantic meaning of the sentences, allowing us to compare and measure the similarity between different sentences. Sentence transformers are trained on large amounts of text data using techniques such as deep learning and transfer learning, enabling them to generate high-quality sentence embeddings. By using sentence transformers, we can effectively cluster and categorize text data based on their semantic similarity, which is useful for tasks such as topic modeling, information retrieval, and text classification. 

#### Dimensionality Reduction
`umap_model`

In [ ]:
if device.type == "cuda":
    from cuml.manifold import UMAP
else:
    from umap import UMAP

umap_model = UMAP(
    n_neighbors=15, # Number of neighboring points used in manifold approximation
    n_components=100, # Number of dimensions in which to approximate the manifold
    min_dist=0.0, # Minimum distance between points in the manifold
    metric='cosine', # Distance metric used in the manifold approximation
    low_memory=False, # This parameter is used to determine whether to use a low memory mode for the UMAP algorithm
    random_state=42 # For reproducibility
)

After having created our numerical representations of the documents we have to reduce the dimensionality of these representations. Cluster models typically have difficulty handling high dimensional data due to the curse of dimensionality.
UMAP is the default method used as it efficiently captures both local and global structures in lower dimensions. It is a technique that can keep some of a dataset's local and global structure when reducing its dimensionality. This structure is important to keep as it contains the information necessary to create clusters of semantically similar documents.

BERTopic is flexible, allowing for the use of alternative dimensionality reduction techniques such as PCA and Truncated SVD. Users can customize these methods by specifying parameters for the desired model and integrate them into BERTopic by replacing the umap_model parameter.

Furthermore, for handling large datasets, cuML offers a GPU-accelerated UMAP implementation. 

Additionally, BERTopic provides an option to bypass dimensionality reduction entirely, using an "empty" model that does not alter the data, simplifying the pipeline to focus directly on clustering and topic representation.

#### Clustering
`hdbscan_model`

In [ ]:
if device.type == "cuda":
    from cuml.cluster import HDBSCAN
else:
    from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=10, # Minimum size of clusters
                        metric='euclidean', # Metric for HDBSCAN
                        cluster_selection_method='eom', # Method for selecting clusters
                        prediction_data=True)# Use prediction data for clustering

After having reduced our embeddings, we can start clustering our data. For that, we leverage a density-based clustering technique, HDBSCAN. It can find clusters of different shapes and of varying densities. It has the nice feature of identifying outliers where possible. As a result, we do not force documents into a cluster where they might not belong. This will improve the resulting topic representation as there is less noise to draw from.

However, BERTopic is designed to be modular, allowing different clustering models to be integrated. This flexibility is achieved through the hdbscan_model parameter, which can accept any clustering model with .fit(), .predict(), and .labels_ attributes.

**Example Clustering Algorithms:**

- **HDBSCAN:**
    - The default clustering method in BERTopic, known for capturing clusters of different densities effectively. Although the original HDBSCAN implementation is highly effective, it may struggle with large datasets. To address this, cuML can be used to accelerate HDBSCAN using GPU, enhancing its performance on larger data.
- **K-means**:
    - An alternative to HDBSCAN, k-Means clustering allows you to specify the number of clusters, ensuring every point is assigned to a cluster, thus eliminating outliers. However, this approach can introduce noise, potentially degrading topic representations. To mitigate this, using vectorizer_model=CountVectorizer(stop_words="english") can significantly improve topic quality.
- **Agglomerative Clustering**:
    - Another option from the sklearn library, Agglomerative Clustering, can also be used. While it may lack a .predict() method, it can still integrate with BERTopic. However, using BERTopic’s .transform() function might result in errors due to this limitation.

### Cluster Refinement and Representation

#### Vectorizer
`vectorizer_model`


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words="english", # Remove stop words
                                   ngram_range=(1, 2), # The lower and upper boundary of the range of n-values for different n-grams to be extracted
                                   min_df=10) # When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold


Before we can start creating the topic representation we first need to select a technique that allows for modularity in BERTopic's algorithm. When we use HDBSCAN as a cluster model, we may assume that our clusters have different degrees of density and different shapes. This means that a centroid-based topic representation technique might not be the best-fitting model. In other words, we want a topic representation technique that makes little to no assumption on the expected structure of the clusters.

To do this, we first combine all documents in a cluster into a single document. That, very long, document then represents the cluster. Then, we can count how often each word appears in each cluster. This generates something called a bag-of-words representation in which the frequency of each word in each cluster can be found. This bag-of-words representation is therefore on a cluster level and not on a document level. This distinction is important as we are interested in words on a topic level (i.e., cluster level). By using a bag-of-words representation, no assumption is made concerning the structure of the clusters. Moreover, the bag-of-words representation is L1-normalized to account for clusters that have different sizes.


**Important Parameters:**
- **ngram_range:**
    The ngram_range parameter allows us to decide how many tokens each entity is in a topic representation. For example, we have words like game and team with a length of 1 in a topic but it would also make sense to have words like hockey league with a length of 2. To allow for these words to be generated, we can set the ngram_range parameter:


- **stop_words:**
    - Stop words are small words such as 'she' or 'the'. Stop words are something we typically want to prevent in our topic representations as they do not give additional information to the topic. To prevent those stop words, we can use the stop_words parameter in the CountVectorizer to remove them from the representations. If 'english', a built-in stop word list for English is used. There are several known issues with 'english' and you should consider an alternative.

        - If a list, that list is assumed to contain stop words, all of which
        will be removed from the resulting tokens.
        Only applies if ``analyzer == 'word'``.

        - If None, no stop words will be used. In this case, setting `max_df`
        to a higher value, such as in the range (0.7, 1.0), can automatically detect
        and filter stop words based on intra corpus document frequency of terms.

- **min_df:**
    - One important parameter to keep in mind is the min_df. This is typically an integer representing how frequent a word must be before being added to our representation. You can imagine that if we have a million documents and a certain word only appears a single time across all of them, then it would be highly unlikely to be representative of a topic. Typically, the c-TF-IDF calculation removes that word from the topic representation but when you have millions of documents, that will also lead to a very large topic-term matrix. To prevent a huge vocabulary, we can set the min_df to only accept words that have a minimum frequency.
    - When you have millions of documents or error issues, I would advise increasing the value of min_df as long as the topic representations might sense. 

- **max_features:**
    - A parameter similar to min_df is max_features which allows you to select the top n most frequent words to be used in the topic representation. Setting this, for example, to 10_000 creates a topic-term matrix with 10_000 terms. This helps you control the size of the topic-term matrix directly without having to fiddle around with the min_df parameter:

#### Weighting
`ctfidf_model`

BERTopic extends Term-Frequency Inverse-Document-Frequency (TF-IDF),to a class-based Term-Frequency Inverse-Document-Frequency (c-TF-IDF). This adaptation of the standard TF-IDF to c-TF-IDF works at the topic level rather than the individual document level, effectively distinguishing what makes the documents within one topic unique compared to documents across other topics. This adjustment enables a more nuanced understanding of how topics differ from one another, refining the entire topic modeling process.

**Default Mathematical Representation:**

For a term $x$ within class $c$:

$$
W_{x,c} = \lVert{f_{x,c}}\rVert\times log(IDF_x + 1)
$$

Where:
- $W_{x,c}$ is the weighting of the term $x$ within class $c$
- $f_{x,c}$ the frequency of word $x$ in class $c$
- $IDF_x$ is the inverse document frequency defined by:
    - $IDF_x = \frac{A}{F_x}$
        - $A$ is the average number of words per class
        - $F_x$ is the frequency of word $x$ across all classes


**How it works:**
1. Each cluster is converted to a single document instead of a set of documents. 

2. We extract the frequency of word $x$ in class $c$, where $c$ refers to the cluster we created before. This results in our class-based $f_{x,c}$ representation. This representation is L1-normalized to account for the differences in topic sizes.

3. We take the logarithm of one plus the average number of words per class $A$ divided by the frequency of word $x$ across all classes. We add one within the logarithm to force values to be positive. This results in our class-based *$IDF$* representation.

4. Then multiply $f$ with $idf$ to get the importance score per word in each class.


**Parameters**
- **reduce_frequent_words:**
    Some words appear quite often in every topic but are generally not considered stop words as found in the CountVectorizer(stop_words="english") list. To further reduce these frequent words, we can use reduce_frequent_words to take the square root of the term frequency after applying the weighting scheme.

    Instead of the default term frequency, we take the square root of the term frequency after normalizing the frequency matrix:


    $$
    \lVert f_{x,c} \rVert \rightarrow \sqrt{\lVert f_{x,c} \rVert}
    $$

    Although seemingly a small change, it can have quite a large effect on the number of stop words in the resulting topic representations. It can be enabled as follows:


- **bm25_weighting:**
    The bm25_weighting is a boolean parameter that indicates whether a class-based BM-25 weighting measure is used instead of the default method.

    $$
    log(\frac{A}{F_x} + 1) \rightarrow log(\frac{A - F_x + 0.5}{F_x + 0.5} + 1)
    $$

    At smaller datasets, this variant can be more robust to stop words that appear in your data. It can be enabled as follows:

#### Topic Representation
`representation_model`

BERTopic offers various representation models that enhance the accuracy and relevance of topic modeling beyond its default Bag-of-Words and c-TF-IDF methods. These models allow for significant fine-tuning of topic representations after initial training, without the need to re-train the model. These are not used by default, and the use of them is mostly for the purposes of understandability.

**KeyBERTInspired**
- One such model is KeyBERTInspired, which refines topics by leveraging c-TF-IDF to create representative documents for each topic. It then measures the semantic relationships between these documents and potential keywords, enhancing the initial keyword sets based on their relevance and coherence with the topic. This model effectively speeds up the inference process while maintaining a high level of accuracy in keyword representation.

**PartOfSpeech (POS)**
- Another model, PartOfSpeech (POS), utilizes part of speech tagging to refine keywords further. It extracts documents containing specific keywords and uses Spacy's POS module to filter and rank these keywords by their c-TF-IDF values. This method helps in identifying more relevant noun phrases and adjectives that better represent the topics.

**Maximal Marginal Relevance (MMR)**
- Finally, Maximal Marginal Relevance (MMR) reduces redundancy within the keywords. MMR selects diverse and representative keywords by balancing their relevance to the document and their similarity to each other, thereby minimizing overlap and enhancing topic distinctiveness.

In [ ]:
from bertopic.representation import KeyBERTInspired

# Create your representation model
representation_model = KeyBERTInspired()

# Data Loading and Representation for Training

## Loading Data

We load all the story data in the format that was previously used 

In [ ]:
# These files are located in the story_data folder of the Box
# We'll build an encoding model using this set of stories for this tutorial.
train_stories = ['adollshouse', 'adventuresinsayingyes', 'alternateithicatom', 'avatar', 'buck', 'exorcism',
            'eyespy', 'fromboyhoodtofatherhood', 'hangtime', 'haveyoumethimyet', 'howtodraw', 'inamoment',
            'itsabox', 'legacy', 'naked', 'odetostepfather', 'sloth',
            'souls', 'stagefright', 'swimmingwithastronauts', 'thatthingonmyarm', 'theclosetthatateeverything',
            'tildeath', 'undertheinfluence']

test_stories = ["wheretheressmoke"]

# Load the data if it hasn't already been loaded. It's been put in a try except to only load it if 
# it's not been defined so we can easily re-run the whole notebook without having to do too much processing

try:
    wordseqs
except NameError:
    grids = joblib.load(grids_path) # Load TextGrids containing story annotations
    trfiles = joblib.load(trfiles_path) # Load TRFiles containing TR information

    # Filter out the other stories for the tutorial
    for story in list(grids):
        if story not in (train_stories + test_stories):
            del grids[story]
            del trfiles[story]

    # Make datasequence for story
    wordseqs = make_word_ds(grids, trfiles)

## Turning the word sequences into shorter document chunks

Ideally, BERTopic wants to classify a large number of whole documents. This includes things like wikipedia articles, newspaper articles, arxiv papers etc. What we have though, is a relatively short list of stories, of long duration. In normal training, the model will take a single document, assign it to embedding space, and then assign it to a cluster. Since the number of stories, is in the order <100, this leads us to not enough topics to be representative of the transitions during the story.

To get around this issue, what we'd like to do is to split up the story into smaller chunks, where a single topic may be better represented. 

In [ ]:
def split_story_into_tr_windows(datasequence):
    """
    Given a datasequence, split the story into windows of size window_size. Each window corresponds with 2 seconds of audio.

    Parameters:
    datasequence (DataSequence): The DataSequence object containing the story data.

    Returns:
    word_sequences (list): A list of strings, where each string is a window of the story.

    Example:
    >>> datasequence = wordseqs['adollshouse']
    >>> split_story_into_tr_windows(datasequence)
    ['nora helmer enters the house with a number of parcels a porter follows her carrying a christmas tree',
    """
    word_sequences = []
    chunks = datasequence.chunks() # gets a list of the chunks in the story
    for chunk in chunks:
        string = ' '.join(chunk)
        if len(string) > 0:
            word_sequences.append(string)
    return word_sequences

In [ ]:
def sliding_window(strings, window_size: int=1, window_overlap = None):
    """
    Generate sliding windows of specified size from a list of strings.

    Parameters:
    strings (list): A list of strings.
    window_size (int): The size of each sliding window.
    window_overlap (int): The overlap between consecutive windows (default is 1).

    Returns:
    list: A list of sliding windows.

    Example:
    >>> sliding_window(['a', 'b', 'c', 'd'], 2)
    ['a b', 'b c', 'c d']
    """

    if not isinstance(window_size, int) or window_size < 1:
        raise ValueError("Window size must be an integer greater than or equal to 1.")
    if window_overlap is None:
        window_overlap = window_size - 1
    if not isinstance(window_overlap, int) or window_overlap < 0 or window_overlap >= window_size:
        raise ValueError("Window overlap must be an integer greater than or equal to 0 and less than window size.")
    
    windows = []
    step = window_size - window_overlap
    
    for i in range(0, len(strings) - window_size + 1, step):
        window = ' '.join(strings[i:i + window_size])
        windows.append(window)
    
    return windows

In [ ]:
# For each story, go through each datasequence and split them into specific windows. Let's default it to 8 seconds (4 TR windows)
window_sizes = [4]

# This could be a list if you wanted to use multiple sliding window sizes

windowed_wordseqs = {}
for window_size in window_sizes:
    for key in wordseqs.keys():
        datasequence = wordseqs[key]
        split_sequence = split_story_into_tr_windows(datasequence)
        windowed_sequence = sliding_window(split_sequence, window_size=window_size)
        dict_name = f"{key}_{window_size}"
        windowed_wordseqs[dict_name] = windowed_sequence

## Precompute the embeddings

In [ ]:
# Define docs for the topic model
docs = []

for key in windowed_wordseqs.keys():
    docs += windowed_wordseqs[key]

# define the embeddings as per best practice
embedding_model = SentenceTransformer(model_name_or_path = "all-MiniLM-L6-v2",device=device)

try:
    embeddings
except NameError:
    embeddings = embedding_model.encode(docs, show_progress_bar=True,device=device)

NameError: name 'SentenceTransformer' is not defined

# Training the Topic Model

All the actual modules have been defined above, so go back up for their definitions

In [35]:
train_new_model = True # If you're running this notebook for the first time, and want to train a topic model, set this to True. Otherwise, set it to False.

if train_new_model:

    embedding_model = embedding_model

    umap_model = UMAP(
        n_neighbors=15, # Number of neighboring points used in manifold approximation
        n_components=2, # Number of dimensions in which to approximate the manifold
        min_dist=0.0, # Minimum distance between points in the manifold
        metric='cosine', # Distance metric used in the manifold approximation
        low_memory=False, # This parameter is used to determine whether to use a low memory mode for the UMAP algorithm
        random_state=42 # For reproducibility
    )

    hdbscan_model = HDBSCAN(min_cluster_size=10, # Minimum size of clusters
                            metric='euclidean', # Metric for HDBSCAN
                            cluster_selection_method='eom', # Method for selecting clusters
                            prediction_data=True)# Use prediction data for clustering
    
    vectorizer_model = CountVectorizer(stop_words="english", 
                                       ngram_range=(1, 2), 
                                       min_df=10)
    
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

    representation_model = [KeyBERTInspired(top_n_words=30), 
                        MaximalMarginalRelevance(diversity=.2),
                        PartOfSpeech("en_core_web_sm")]
    
    topic_model = BERTopic(embedding_model=embedding_model, # The SentenceTransformer model
                            umap_model=umap_model, # The UMAP model
                            hdbscan_model=hdbscan_model,
                            vectorizer_model=vectorizer_model,
                            ctfidf_model=ctfidf_model,
                            calculate_probabilities=True, # Calculate the probabilities of each topic
                            verbose=True,
                            nr_topics="auto") 
else:
    pass

# Fit the topic model
topics, probs = topic_model.fit_transform(documents = docs, 
                                          embeddings=embeddings)

display(topic_model.get_topic_info())
topic_model.visualize_topics()

2024-07-15 20:17:30,339 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-15 20:17:37,995 - BERTopic - Dimensionality - Completed ✓
2024-07-15 20:17:37,996 - BERTopic - Cluster - Start clustering the reduced embeddings
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2024-07-15 20:19:14,184 - BERTopic - Cluster - Completed ✓
2024-07-15 20:19:14,185 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-07-15 20:19:14,630 - BERTopic - Representation - Completed ✓
2024-07-15 20:19:14,631 - BERTopic - Topic reduction - Reducing number of topics
2024-07-15 20:19:15,192 - BERTopic - Topic reduction - Reduced number of topics from 640 to 39


,Topic,Count,Name,Representation,Representative_Docs
0,-1,1906,-1_years_thing_say_little,"[years, thing, say, little, time, um, really, ...",[and and through uh a little bit of what what ...
1,0,14694,0_years_thing_little_say,"[years, thing, little, say, time, really, uh, ...",[now here's what they don't tell you about hav...
2,1,149,1_say_thing_really_uh,"[say, thing, really, uh, um, know, just, like,...",[i i was originally asked by the moth to do th...
3,2,53,2_time_say_just_know,"[time, say, just, know, like, , , , , ]",[they say it works every time a quantum event ...
4,3,36,3_just_really__,"[just, really, , , , , , , , ]",[that i needed to photocopy i opened the machi...
5,4,35,4_say_just_know_little,"[say, just, know, little, like, , , , , ]",[i mean you're just describing a a horrible de...
6,5,30,5_little_thing_say_like,"[little, thing, say, like, know, um, , , , ]",[are always against thinking about things in t...
7,6,26,6_say___,"[say, , , , , , , , , ]",[ethel looks drawn and then about an hour late...
8,7,25,7_say___,"[say, , , , , , , , , ]",[what happened vijay weren't we driving vijay ...
9,8,25,8_know_really_just_like,"[know, really, just, like, , , , , , ]",[about something called actuarial science for ...


## Refining the representations

### Outlier Reduction

In [37]:
# Use the "c-TF-IDF" strategy with a threshold
new_topics = topic_model.reduce_outliers(docs, topics , probabilities=probs, strategy="probabilities")

# Reduce all outliers that are left with the "distributions" strategy
new_topics = topic_model.reduce_outliers(docs, new_topics, strategy="probabilities", probabilities = probs)

topic_model.update_topics(docs, topics=new_topics)
display(topic_model.get_topic_info())
topic_model.visualize_topics()

2024-07-15 20:19:41,101 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


,Topic,Count,Name,Representation,Representative_Docs
0,0,16598,0_and_the_to_was,"[and, the, to, was, you, that, my, it, of, like]",[now here's what they don't tell you about hav...
1,1,149,1_sloth_sin_acedia_word,"[sloth, sin, acedia, word, deadly, it, as, of,...",[i i was originally asked by the moth to do th...
2,2,53,2_universes_universe_alternate_quantum,"[universes, universe, alternate, quantum, tail...",[they say it works every time a quantum event ...
3,3,36,3_machine_photocopying_papers_duty,"[machine, photocopying, papers, duty, files, a...",[that i needed to photocopy i opened the machi...
4,4,35,4_todd_nihilism_hearing_wait,"[todd, nihilism, hearing, wait, negativity, du...",[i mean you're just describing a a horrible de...
5,5,30,5_sin_shrink_guilt_is,"[sin, shrink, guilt, is, doesn, modification, ...",[are always against thinking about things in t...
6,6,26,6_rose_died_sixty_aunt,"[rose, died, sixty, aunt, thursday, calculatio...",[ethel looks drawn and then about an hour late...
7,7,25,7_vijay_mantra_temple_mother,"[vijay, mantra, temple, mother, weren, driving...",[what happened vijay weren't we driving vijay ...
8,8,25,8_use_mortality_rates_areas,"[use, mortality, rates, areas, invest, hence, ...",[about something called actuarial science for ...
9,9,23,9_neuroscience_neuro_exists_pbs,"[neuroscience, neuro, exists, pbs, nova, progr...",[it makes me think about a moment in time when...


## Saving a Model

# Running a model on the full dataset

## Load the data

In [4]:
# #### Dependencies ####
# import numpy as np
# import logging
# import os
# import sys
# import time
import joblib
# import matplotlib.pyplot as plt
# import torch
# import json
# import cortex # This dependency is pycortex, which enables the plotting of flatmaps. It can be disabled.
# from cvxopt import matrix, solvers # Only necessary for the stacked model.
# from transformers import AutoTokenizer, AutoModelForCausalLM # Only necessary for feature extraction.
# import subprocess
# from tqdm.autonotebook import tqdm, trange

# # Repository imports
# from ridge_utils.ridge import bootstrap_ridge
# import ridge_utils.npp
# from ridge_utils.util import make_delayed
from ridge_utils.dsutils import make_word_ds
from ridge_utils.DataSequence import DataSequence
# from ridge_utils.tokenization_helpers import generate_efficient_feat_dicts_opt
# from ridge_utils.tokenization_helpers import convert_to_feature_mats_opt

# # Topic model imports
# from bertopic import BERTopic
# from sentence_transformers import SentenceTransformer # For generating embeddings
# from sklearn.decomposition import PCA # To speed up the UMAP
# from sklearn.feature_extraction.text import CountVectorizer 
# from bertopic.vectorizers import ClassTfidfTransformer
# try:
#     import en_core_web_sm
# except ModuleNotFoundError:
#     !python -m spacy download en_core_web_sm
#     import en_core_web_sm
# from bertopic.representation import PartOfSpeech, KeyBERTInspired, MaximalMarginalRelevance


# These files are located in the story_data folder of the Box
grids_path = "../data/story_data/grids_huge.jbl"
trfiles_path = "../data/story_data/trfiles_huge.jbl"
grids = joblib.load(grids_path) # Load TextGrids containing story annotations
trfiles = joblib.load(trfiles_path) # Load TRFiles containing TR information


# Make datasequence for story
wordseqs = make_word_ds(grids, trfiles)

# For each story, go through each datasequence and split them into specific windows. Let's default it to 8 seconds (4 TR windows)
window_sizes = [4]

# This could be a list if you wanted to use multiple sliding window sizes

windowed_wordseqs = {}
for window_size in window_sizes:
    for key in wordseqs.keys():
        datasequence = wordseqs[key]
        split_sequence = split_story_into_tr_windows(datasequence)
        windowed_sequence = sliding_window(split_sequence, window_size=window_size)
        dict_name = f"{key}_{window_size}"
        windowed_wordseqs[dict_name] = windowed_sequence

# Define docs for the topic model
docs = []

for key in windowed_wordseqs.keys():
    docs += windowed_wordseqs[key]

NameError: name 'split_story_into_tr_windows' is not defined

In [44]:
# Define the embedding model and then embed
embedding_model = SentenceTransformer(model_name_or_path = "all-MiniLM-L6-v2",device=device)
embeddings = embedding_model.encode(docs, show_progress_bar=True,device=device)

Batches:   0%|          | 0/275 [00:00<?, ?it/s]

In [45]:
umap_model = UMAP(
    n_neighbors=15, # Number of neighboring points used in manifold approximation
    n_components=2, # Number of dimensions in which to approximate the manifold
    min_dist=0.0, # Minimum distance between points in the manifold
    metric='cosine', # Distance metric used in the manifold approximation
    low_memory=False, # This parameter is used to determine whether to use a low memory mode for the UMAP algorithm
    random_state=42 # For reproducibility
)

hdbscan_model = HDBSCAN(min_cluster_size=10, # Minimum size of clusters
                        metric='euclidean', # Metric for HDBSCAN
                        cluster_selection_method='eom', # Method for selecting clusters
                        prediction_data=True)# Use prediction data for clustering

vectorizer_model = CountVectorizer(stop_words="english", 
                                    ngram_range=(1, 2), 
                                    min_df=10)

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

representation_model = [KeyBERTInspired(top_n_words=30), 
                    MaximalMarginalRelevance(diversity=.2),
                    PartOfSpeech("en_core_web_sm")]

topic_model = BERTopic(embedding_model=embedding_model, # The SentenceTransformer model
                        umap_model=umap_model, # The UMAP model
                        hdbscan_model=hdbscan_model,
                        vectorizer_model=vectorizer_model,
                        ctfidf_model=ctfidf_model,
                        calculate_probabilities=True, # Calculate the probabilities of each topic
                        verbose=True,
                        nr_topics="auto") 

# Fit the topic model
topics, probs = topic_model.fit_transform(documents = docs, 
                                          embeddings=embeddings)

display(topic_model.get_topic_info())
topic_model.visualize_topics()

2024-07-15 20:28:43,594 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-07-15 20:28:55,144 - BERTopic - Dimensionality - Completed ✓
2024-07-15 20:28:55,144 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-07-15 20:29:05,351 - BERTopic - Cluster - Completed ✓
2024-07-15 20:29:05,352 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-07-15 20:29:05,571 - BERTopic - Representation - Completed ✓
2024-07-15 20:29:05,571 - BERTopic - Topic reduction - Reducing number of topics
2024-07-15 20:29:05,761 - BERTopic - Topic reduction - Reduced number of topics from 213 to 91


,Topic,Count,Name,Representation,Representative_Docs
0,-1,2398,-1_um uh_saw_point_suddenly,"[um uh, saw, point, suddenly, okay, thinking, ...",[all the time you're acting like a little girl...
1,0,3997,0_school_wanna_year_ask,"[school, wanna, year, ask, stuff, called, away...",[you'll have no problem she's like it's not th...
2,1,170,1_met_looks_person_head,"[met, looks, person, head, better, walk, room,...",[if i'm gonna be the per kind of person who ma...
3,2,110,2_does_need_telling_long,"[does, need, telling, long, met, started, peop...",[does she need jesus does she need mary kay or...
4,3,93,3_seven_probably_does_hear,"[seven, probably, does, hear, saw, way, actual...",[a college dropout i don't know how to pronoun...
...,...,...,...,...,...
86,85,11,85_um uh_maybe_thought_uh,"[um uh, maybe, thought, uh, um, just, , , , ]",[and they couldn't figure out the jury's still...
87,86,11,86_comes_want_didn_says,"[comes, want, didn, says, life, did, tell, sai...",[comes over to me and says well ms vega we did...
88,87,11,87_long_make_ll_don,"[long, make, ll, don, left, probably, way, tel...",[you can tell a kid that's not gonna survive l...
89,88,11,88_ve_met_life_new,"[ve, met, life, new, day, thought, , , , ]",[with my meal not them i'm not saying i've nev...
